In [60]:
import pandas as pd

# Load the FAQ data
documents = pd.read_csv('../data/faqs.csv')
documents = documents.drop_duplicates(subset=['Question', 'Answer', "Category"], keep='first')

documents.head()
documents.shape

(429, 5)

In [61]:
import hashlib

def generate_document_id(doc):
    combined = f"{doc['Category']}-{doc['Question']}-{doc['Answer'][:10]}-{doc['Answer'][:-10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [62]:
def generate_document_ids_for_dataframe(df):
    df['id'] = df.apply(generate_document_id, axis=1)
    return df

In [63]:
documents = generate_document_ids_for_dataframe(documents)

In [64]:
documents

,Category,Subcategory,Question,Answer,URL,id
0,General/other,FAQ BMBF-Förderung,Umfassen die Ausgaben für Literatur in der Position F0840 auch die kostenpflichtige Nutzung von Literaturdatenbanken?,"Wenn die kostenpflichtige Nutzung von Literaturdatenbanken für die Durchführung des Projektes erforderlich ist, sind die Ausgaben dafür zuwendungsfähig.",https://vdivde-it.de/de/umfassen-die-ausgaben-fuer-literatur-der-position-f0840-auch-die-kostenpflichtige-nutzung-von,604174c6
1,General/other,FAQ BMBF-Förderung,Ab welchem Betrag müssen Geräte abgeschrieben werden und bis zu welchem Betrag können Geräte ähnlich wie Verbrauchsmaterial bestellt werden?,Bitte benutzen Sie zur Beantwortung dieser Frage die AfA-Tabellen des Bundesfinanzministeriums.,https://vdivde-it.de/de/ab-welchem-betrag-muessen-geraete-abgeschrieben-werden-und-bis-zu-welchem-betrag-koennen-geraete,904dc228
2,General/other,FAQ BMBF-Förderung,Ab welcher Grenze müssen bei einer Beauftragung mehrere Angebote eingeholt werden?,"Der Projektträger kann an dieser Stelle leider keine Vergaberechtsberatung durchführen.Bei der Vergabe eines Auftrages sind die einschlägigen Bestimmungen der Unterschwellenvergabeordnung (UVgO), der Vergabeordnung VgV, des Gesetzes gegen Wettbewerbsbeschränkung (GWB) sowie die entsprechenden Regelungen in den NABF zu beachten und die Notwendigkeit eines Vergabeverfahrens zu prüfen.",https://vdivde-it.de/de/ab-welcher-grenze-muessen-bei-einer-beauftragung-mehrere-angebote-eingeholt-werden,5e60e572
3,General/other,FAQ BMBF-Förderung,Ab wieviel Mitarbeitern gilt eine Firma als Großunternehmen und nicht mehr als mittelgroßes Unternehmen?,Für die Einstufung als KMU ( meist ab 250 Personen) ist neben der Anzahl der Mitarbeiter auch die Höhe der Bilanzsumme oder des Jahresumsatzes zu berücksichtigen.,https://vdivde-it.de/de/ab-wieviel-mitarbeitern-gilt-eine-firma-als-grossunternehmen-und-nicht-mehr-als-mittelgrosses,c9a66a1d
4,General/other,FAQ BMBF-Förderung,An wen kann ich mich für eine Antragsberatung wenden?,Ansprechpersonen für eine Antragsberatung sind der Förderbekanntmachung zu entnehmen.,https://vdivde-it.de/de/wen-kann-ich-mich-fuer-eine-antragsberatung-wenden,dd1dcd6d
...,...,...,...,...,...,...
1041,Verwendungsnachweis,Verwendungsnachweis,Wo findet man die Vorlage für die Belegliste?,Die Vorlage für die Belegliste finden Sie im Formularschrank des Bundes. (siehe https://foerderportal.bund.de/easy/module/profi_formularschrank/download.php?datei1=188) Wir empfehlen Hochschulen folgende Vorlage zur Belegliste (siehe https://vdivde-it.de/de/media/717) zu verwenden. Hier werden Sie bei der Abrechnung der Jahressonderzahlung unterstützt.,https://vdivde-it.de/de/wo-findet-man-die-vorlage-fuer-die-belegliste,1b0acdc6
1042,Verwendungsnachweis,Verwendungsnachweis,Wo kann ich die Unterlagen zum Verwendungsnachweis hochladen?,Der zahlenmäßige Teil des Verwendungsnachweises kann in profi-Online erstellt werden. Die Anlagen dazu und der Sachbericht können über profi-Online mit hochgeladen werden. Große Dateien können Sie uns ggf. auch über unsere sichere Datenübertragung (siehe https://www.vdivde-it.de/tools/ ) zusenden.,https://vdivde-it.de/de/wo-kann-ich-die-unterlagen-zum-verwendungsnachweis-hochladen,0097e4ae
1043,Verwendungsnachweis,Verwendungsnachweis,"Wo kann ich einsehen, welche Mittel gesperrt sind?","Im Zuwendungsbescheid werden die Sperren und die Auflagen, die zur Entsperrung zu erfüllen sind, angegeben. Die Sperren werden dann auch in der Vorkalkulation/ im Gesamtfinanzierungsplan sind im unteren Teil ausgewiesen. Für die Entsperrung ist ein schriftlichen Antrag zu stellen. Gesperrte Mittel sind von der Umwidmung in andere Positionen ausgeschlossen.",https://vdivde-it.de/de/wo-kann-ich-einsehen-welche-mittel-gesperrt-sind,1f498485
1044,Verwendungsnachweis,Verwendungsnachweis,"Wofür darf die Projektpauschale, die Hochschulen erhalten, verwendet werden?",Die Projektpauschale dient zur Teilfinanzierung der durch das jeweilige Forschungsprojekt

In [65]:
# Assuming 'documents' is your DataFrame
import pandas as pd
import json

# Convert the DataFrame to a list of dictionaries
data = documents.to_dict(orient='records')

# Save as a JSON file
json_file_path = 'documents-with-ids.json'
with open(json_file_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f'Data saved to {json_file_path}')


Data saved to documents-with-ids.json


In [66]:
from collections import defaultdict

hashes = defaultdict(list)

for _, doc in documents.iterrows():
    doc_id = doc['id']
    hashes[doc_id].append(doc)


In [67]:
len(hashes), len(documents)


(429, 429)

In [68]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

In [69]:
hashes["3bf179c3"]

hashes["776af204"][0]

Category                                                                                                                                                                                                                                                                                              Antragstellung
Subcategory                                                                                                                                                                                                                                                                                           Antragstellung
Question                                                                                                                                                                                                                                              Dürfen die Vorhabenbeschreibungen in Englisch verfasst werden?
Answer          Vorhabenbeschreibungen sind in deutscher Sprache zu verfa

In [70]:
pd.set_option('display.max_colwidth', None)

documents[documents["id"]=="776af204"]

,Category,Subcategory,Question,Answer,URL,id
222,Antragstellung,Antragstellung,Dürfen die Vorhabenbeschreibungen in Englisch verfasst werden?,"Vorhabenbeschreibungen sind in deutscher Sprache zu verfassen. Ausnahmen können Projekte bilden, die im Rahmen einer europäischen Maßnahme durchgeführt werden. Nähere Informationen zu den Anforderungen entnehmen Sie bitte der betreffenden im Bundesanzeiger veröffentlichten Förderrichtlinie.",https://vdivde-it.de/de/duerfen-die-vorhabenbeschreibungen-englisch-verfasst-werden,776af204


In [72]:
prompt_template = """
Du simulierst einen Kunden von VDI-VDE-IT, der den Chatbot nutzen möchte, um Fragen zu stellen.
Formuliere 5 alternative Fragen, die dieser Kunde auf Basis des FAQ-Eintrags stellen könnte. Die Fragen sollten vollständig und nicht zu kurz sein.
Verwende, wenn möglich, so wenige Wörter wie möglich aus dem Eintrag. Als Kontext, um die Fragen präziser zu formulieren, kannst du Hintergründe zur Arbeit von VDI-VDE/IT und anderen Projektträgern heranziehen, die du kennst.

Der Eintrag:

Frage: {Question}
Antwort: {Answer}
Kontext / Kategorie: {Category}

Gib das Ergebnis als parsierbares JSON ohne Codeblöcke aus:

["frage1", "frage2", ..., "frage5"]
""".strip()


In [73]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


from openai import OpenAI
client = OpenAI()

# Load your OpenAI API key
client.api_key = os.getenv("OPENAI_API_KEY")

In [74]:

def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [75]:
from tqdm.auto import tqdm


/usr/local/anaconda3/envs/vdi_chatbot_py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [76]:
results = {}


In [77]:
from tqdm import tqdm

for _, doc in tqdm(documents.iterrows(), total=len(documents)): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions


  0%|          | 0/429 [00:00<?, ?it/s]

100%|██████████| 429/429 [10:56<00:00,  1.53s/it]


In [78]:
import pickle

# Assuming 'results' is the dictionary containing your generated questions
# Replace 'results' with your actual variable

file_path = 'generated_questions.bin'

with open(file_path, 'wb') as f:
    pickle.dump(results, f)

print(f'Results saved to {file_path}')


Results saved to generated_questions.bin


In [79]:
with open('generated_questions.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [80]:
import json
parsed_results = {}

for doc_id, json_questions in results.items():
    parsed_results[doc_id] = json.loads(json_questions)

In [81]:
doc_index = {doc['id']: doc for _, doc in documents.iterrows()}
final_results = []

for doc_id, questions in parsed_results.items():
    category = doc_index[doc_id]['Category']
    for q in questions:
        final_results.append((q, category, doc_id))

In [82]:
import pandas as pd


In [83]:
final_results

[('Sind Ausgaben für kostenpflichtige Literaturdatenbanken förderfähig?',
  'General/other',
  '604174c6'),
 ('Gibt es finanzielle Unterstützung für den Zugriff auf Literaturdatenbanken?',
  'General/other',
  '604174c6'),
 ('Kann ich die Kosten für Datenbanken in mein Projektbudget aufnehmen?',
  'General/other',
  '604174c6'),
 ('Was passiert, wenn ich Literaturdatenbanken für mein Projekt nutzen muss?',
  'General/other',
  '604174c6'),
 ('Sind nicht kostenlose Datenbanken auch förderfähig bei Projekten?',
  'General/other',
  '604174c6'),
 ('Was ist der Mindestbetrag für die Abschreibung von Geräten?',
  'General/other',
  '904dc228'),
 ('Bis zu welcher Summe können Geräte als Verbrauchsmaterial erworben werden?',
  'General/other',
  '904dc228'),
 ('Wo finde ich die AfA-Tabellen zur Geräteabschreibung?',
  'General/other',
  '904dc228'),
 ('Welche Kriterien gelten für die Klassifizierung von Geräten als Verbrauchsmaterial?',
  'General/other',
  '904dc228'),
 ('Ab wann ist eine Ab

In [84]:
df = pd.DataFrame(final_results, columns=['Question', 'Category', 'Document'])


In [85]:
df.head()

,Question,Category,Document
0,Sind Ausgaben für kostenpflichtige Literaturdatenbanken förderfähig?,General/other,604174c6
1,Gibt es finanzielle Unterstützung für den Zugriff auf Literaturdatenbanken?,General/other,604174c6
2,Kann ich die Kosten für Datenbanken in mein Projektbudget aufnehmen?,General/other,604174c6
3,"Was passiert, wenn ich Literaturdatenbanken für mein Projekt nutzen muss?",General/other,604174c6
4,Sind nicht kostenlose Datenbanken auch förderfähig bei Projekten?,General/other,604174c6


In [86]:
df.to_csv('ground-truth-data.csv', index=False)
